<a href="https://colab.research.google.com/github/Woongheelee7/Leon/blob/main/Liear_Regression_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [224]:
from typing import List
from typing import Tuple
from typing import Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from tqdm import tqdm

sns.set(font_scale=1.5)
sns.set_style("whitegrid", {'grid.linestyle':'--'})

## Reading data

In [225]:
data = pd.read_csv("https://raw.githubusercontent.com/changyaochen/MECE4520/master/data/simple_linear_regression.csv",names=["firstcol","secondcol"])
data['thirdcol']=np.nan
data['thirdcol']=data['firstcol']*data['firstcol']
data.head()



,firstcol,secondcol,thirdcol
0,-15.000,-555.482,225.000000
1,-14.824,-388.301,219.750976
2,-14.648,-317.445,214.563904
3,-14.472,-412.942,209.438784
4,-14.296,-310.663,204.375616


## Simple linear regression

In [226]:


y = data["secondcol"]
x = data["firstcol"]

def simple_linear_regression(
    x: Union[List, np.ndarray, pd.Series], 
    y: Union[List, np.ndarray, pd.Series]) -> Tuple[float, float]:
    """Return the intercept and slope of a simple linear regression."""
    beta_1 = np.cov(x, y)[0][1] / np.cov(x, x)[0][1]
    beta_0 = np.mean(y) - beta_1 * np.mean(x)
    
    return beta_0, beta_1

beta_0, beta_1 = simple_linear_regression(x=x, y=y)

# calculate R^2
y_pred = beta_0 + beta_1 * x
SST = np.sum(np.square(y - np.mean(y)))
residual = y - y_pred
SSE = np.sum(np.square(residual))
r2 = 1 - SSE / SST

print(f"beta_0 is: {beta_0:5.4f}")
print(f"beta_1 is: {beta_1:5.4f}")
print(f"R-square is: {r2:5.4f}")


beta_0 is: -35.3330
beta_1 is: 18.2241
R-square is: 0.8221


In [227]:
# confidence intervals
SE_beta_0 = (np.var(residual, ddof=2) * (1. / len(x) + (np.mean(x))**2 / np.sum((x - np.mean(x))**2)))**0.5
SE_beta_1 = (np.var(residual,ddof=2) / np.sum((x - np.mean(x))**2))**0.5 

print(f"The standard error for beta_0 is: {SE_beta_0:5.4f}")
print(f"The standard error for beta_1 is: {SE_beta_1:5.4f}")

The standard error for beta_0 is: 6.3000
The standard error for beta_1 is: 0.6024


In [228]:
# simple linear regression with the `statsmodels` library
model_1 = smf.ols(formula=' secondcol ~ firstcol',data=data)
result_1 = model_1.fit()
print(result_1.summary())

                            OLS Regression Results                            
Dep. Variable:              secondcol   R-squared:                       0.822
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     915.1
Date:                Thu, 13 Oct 2022   Prob (F-statistic):           3.61e-76
Time:                        03:03:13   Log-Likelihood:                -1174.8
No. Observations:                 200   AIC:                             2354.
Df Residuals:                     198   BIC:                             2360.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -35.3330      6.300     -5.608      0.0

In [229]:
# 1-5 problem.

print("When x=10, CI of y hat")
LOW_pred_y = (beta_0+10*beta_1)-1.96*(np.var(residual, ddof=2) * (1. / len(x) + (10-np.mean(x))**2 / np.sum((x - np.mean(x))**2)))**0.5
UP_pred_y = (beta_0+10*beta_1)+1.96*(np.var(residual, ddof=2) * (1. / len(x) + (10-np.mean(x))**2 / np.sum((x - np.mean(x))**2)))**0.5
print(f"CI with variable [ {LOW_pred_y:5.4f}",f", {UP_pred_y:5.4f}]")
print(beta_0+10*beta_1,"= y hat")
LOW_pred_y2 = (-35.3330+10*18.2241)-1.96*(np.var(residual, ddof=2) * (1. / len(x) + (10-np.mean(x))**2 / np.sum((x - np.mean(x))**2)))**0.5
UP_pred_y2 = (-35.3330+10*18.2241)+1.96*(np.var(residual, ddof=2) * (1. / len(x) + (10-np.mean(x))**2 / np.sum((x - np.mean(x))**2)))**0.5
print(f"CI with constant [ {LOW_pred_y2:5.4f}",f", {UP_pred_y2:5.4f}]")
print(-35.3330+10*18.2241,"= y hat")

When x=10, CI of y hat
CI with variable [ 132.0020 , 161.8136]
146.9077810655238 = y hat
CI with constant [ 132.0022 , 161.8138]
146.908 = y hat


## Multi-variant linear regression

In [230]:
model_2 = smf.ols(formula='secondcol ~ firstcol + thirdcol', data=data)
result_2 = model_2.fit()
print(result_2.summary())

                            OLS Regression Results                            
Dep. Variable:              secondcol   R-squared:                       0.822
Model:                            OLS   Adj. R-squared:                  0.820
Method:                 Least Squares   F-statistic:                     455.4
Date:                Thu, 13 Oct 2022   Prob (F-statistic):           1.33e-74
Time:                        03:03:13   Log-Likelihood:                -1174.8
No. Observations:                 200   AIC:                             2356.
Df Residuals:                     197   BIC:                             2365.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -37.0521      9.020     -4.108      0.0